# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    PROMPT_KGE_DIMENSION,
    INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    ROOT,
)
from llm_manager import (
    VanillaBertClassifier,
)

In [2]:
EPOCHS = 8
BATCH_SIZE_KGE = 128000
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_prompt = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    kge_dimension=PROMPT_KGE_DIMENSION,
    force_recompute=False,
)
graph_representation_generator_input_embeds_replace = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    force_recompute=False,
)

loading pretrained model
Device: 'cpu'
loading pretrained model
Device: 'cpu'


Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
VANILLA_ROOT = f"{ROOT}/llm/vanilla"

In [7]:
vanilla_bert_classifier = VanillaBertClassifier(
    kg_manager.llm_df,
    kg_manager.source_df,
    kg_manager.target_df,
    root_path=VANILLA_ROOT,
)

Some weights of BertForSequenceClassificationRanges were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
kg_manager.llm_df

,source_id,target_id,id_x,id_y,prompt_feature_title,prompt_feature_genres,split,labels
0,0,16,0,16,Sense and Sensibility (1995),"['Drama', 'Romance']",train,1
1,0,28,0,28,"City of Lost Children, The (Cité des enfants p...","['Adventure', 'Drama', 'Fantasy', 'Mystery', '...",val,1
2,0,29,0,29,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,"['Crime', 'Drama']",train,1
3,0,31,0,31,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"['Mystery', 'Sci-Fi', 'Thriller']",test,1
4,0,33,0,33,Babe (1995),"['Children', 'Drama']",train,1
...,...,...,...,...,...,...,...,...
14080084,200947,14478,200947,14478,How to Train Your Dragon (2010),"['Adventure', 'Animation', 'Children', 'Fantas...",train,1
14080085,200947,14811,200947,14811,"A-Team, The (2010)","['Action', 'Comedy', 'Thriller']",train,1
14080086,200947,15058,200947,15058,Scott Pilgrim vs. the World (2010),"['Action', 'Comedy', 'Fantasy', 'Musical', 'Ro...",train,1
14080087,200947,15077,200947,15077,Centurion (2010),"['Action', 'Adventure', 'Drama', 'Thriller', '...",val,1


Next we generate a vanilla llm dataset and tokenize it for training.

In [9]:
dataset_vanilla = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function,
)

Loading dataset from disk:   0%|          | 0/26 [00:00<?, ?it/s]

In [10]:
dataset_vanilla["train"][0]["prompt"]

"0[SEP]16[SEP]Sense and Sensibility (1995)[SEP]['Drama', 'Romance']"

In [11]:
dataset_vanilla

DatasetDict({
    train: Dataset({
        features: ['source_id', 'target_id', 'id_x', 'id_y', 'prompt_feature_title', 'prompt_feature_genres', 'split', 'labels', 'prompt', 'input_ids', 'attention_mask', 'semantic_positional_encoding'],
        num_rows: 7680049
    })
    val: Dataset({
        features: ['source_id', 'target_id', 'id_x', 'id_y', 'prompt_feature_title', 'prompt_feature_genres', 'split', 'labels', 'prompt', 'input_ids', 'attention_mask', 'semantic_positional_encoding'],
        num_rows: 3200020
    })
    test: Dataset({
        features: ['source_id', 'target_id', 'id_x', 'id_y', 'prompt_feature_title', 'prompt_feature_genres', 'split', 'labels', 'prompt', 'input_ids', 'attention_mask', 'semantic_positional_encoding'],
        num_rows: 3200020
    })
})

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [12]:
vanilla_bert_classifier.train_model_on_data(
    dataset_vanilla, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)

  0%|          | 0/240008 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.6752, 'grad_norm': 0.6788515448570251, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 0.6729, 'grad_norm': 0.6454486846923828, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 0.6711, 'grad_norm': 0.5040226578712463, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 0.6677, 'grad_norm': 0.7642132043838501, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.665, 'grad_norm': 0.5805767774581909, 'learning_rate': 5e-06, 'epoch': 0.0}
{'loss': 0.665, 'grad_norm': 0.4583839178085327, 'learning_rate': 6e-06, 'epoch': 0.0}
{'loss': 0.6611, 'grad_norm': 0.545752763748169, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.656, 'grad_norm': 0.5415980815887451, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6479, 'grad_norm': 0.5593803524971008, 'learning_rate': 9e-06, 'epoch': 0.0}
{'loss': 0.6428, 'grad_norm': 0.7430774569511414, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 0.6398, 'grad_norm': 0.6995664238

  0%|          | 0/12501 [00:00<?, ?it/s]

{'eval_loss': 0.1569368988275528, 'eval_accuracy': 0.9476297023143605, 'eval_runtime': 1886.6584, 'eval_samples_per_second': 1696.131, 'eval_steps_per_second': 6.626, 'epoch': 1.0}
{'loss': 0.2146, 'grad_norm': 1.343825101852417, 'learning_rate': 4.3839454214473005e-05, 'epoch': 1.0}
{'loss': 0.2243, 'grad_norm': 1.249559760093689, 'learning_rate': 4.383736660153315e-05, 'epoch': 1.0}
{'loss': 0.2107, 'grad_norm': 0.9336071610450745, 'learning_rate': 4.3835278988593285e-05, 'epoch': 1.0}
{'loss': 0.2183, 'grad_norm': 0.8766353726387024, 'learning_rate': 4.383319137565343e-05, 'epoch': 1.0}
{'loss': 0.223, 'grad_norm': 1.0443757772445679, 'learning_rate': 4.3831103762713565e-05, 'epoch': 1.0}
{'loss': 0.2231, 'grad_norm': 1.2930718660354614, 'learning_rate': 4.382901614977371e-05, 'epoch': 1.0}
{'loss': 0.221, 'grad_norm': 1.6250282526016235, 'learning_rate': 4.3826928536833844e-05, 'epoch': 1.0}
{'loss': 0.2116, 'grad_norm': 0.9932293891906738, 'learning_rate': 4.382484092389398e-05, '

...

  0%|          | 0/12501 [00:00<?, ?it/s]

{'eval_loss': 0.12002688646316528, 'eval_accuracy': 0.9637721014243661, 'eval_runtime': 1882.822, 'eval_samples_per_second': 1699.587, 'eval_steps_per_second': 6.64, 'epoch': 7.0}
{'loss': 0.1908, 'grad_norm': 0.9746504426002502, 'learning_rate': 6.2624212969921675e-06, 'epoch': 7.0}
{'loss': 0.1891, 'grad_norm': 0.8590642213821411, 'learning_rate': 6.260333684052308e-06, 'epoch': 7.0}
{'loss': 0.1756, 'grad_norm': 1.0632327795028687, 'learning_rate': 6.2582460711124474e-06, 'epoch': 7.0}
{'loss': 0.1831, 'grad_norm': 2.1706345081329346, 'learning_rate': 6.2561584581725866e-06, 'epoch': 7.0}
{'loss': 0.1922, 'grad_norm': 0.7817360162734985, 'learning_rate': 6.254070845232727e-06, 'epoch': 7.0}
{'loss': 0.1885, 'grad_norm': 1.1110081672668457, 'learning_rate': 6.251983232292867e-06, 'epoch': 7.0}
{'loss': 0.2014, 'grad_norm': 0.8938820362091064, 'learning_rate': 6.249895619353007e-06, 'epoch': 7.0}
{'loss': 0.1924, 'grad_norm': 0.9694369435310364, 'learning_rate': 6.247808006413147e-06,

  0%|          | 0/12501 [00:00<?, ?it/s]

{'eval_loss': 0.12074045836925507, 'eval_accuracy': 0.9636736645395966, 'eval_runtime': 1879.1551, 'eval_samples_per_second': 1702.904, 'eval_steps_per_second': 6.652, 'epoch': 8.0}
{'train_runtime': 84796.3689, 'train_samples_per_second': 724.564, 'train_steps_per_second': 2.83, 'train_loss': 0.2070751559970371, 'epoch': 8.0}


AttributeError: 'VanillaBertClassifier' object has no attribute 'device'

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.